# Reference file for quantification of extreme weather impacts

https://www.energy.gov/sites/prod/files/2019/09/f67/Oak%20Ridge%20National%20Laboratory%20EIS%20Response.pdf

Explore wildfires, extreme heat (max, median & min), sea-level rise, and flooding.
Infrastructure: generation, substations, transmission lines, & EV chargers.
Counties and population groups most affected?

Identify which infrastructure is impacted?
Graph lost capacity for generation plants; derated capacity for transmission lines
Evaluate accessibility after flooding using methodology in Puerto Rico paper https://www.researchgate.net/publication/362089738_Geospatial_Assessment_Methodology_to_Estimate_Power_Line_Restoration_Access_Vulnerabilities_After_a_Hurricane_in_Puerto_Rico


Importing FEMA flood plain data: 100 year, 200 year and 500 year

In [6]:
#Importing necessary libraries
import pandas as pd
import geopandas as gpd
from arcgis.features import FeatureLayer
from arcgis.features import GeoAccessor
#import arcpy
from arcgis import GIS
gis = GIS()

Note: there are several layers available for the 100 year, 200 year and 500 year flood plains. To add an extra data wrangling step, we import all the available layers and dissolve them in one layer for each category below.

### Importing and combining 100 year flood plain data

In [7]:
#Setting urls for the FEMA feature layers - to change to for loop later

floodrisk_100_year_url_FEMA = 'https://gis.water.ca.gov/arcgis/rest/services/Boundaries/BAM/MapServer/5' #FEMA Effective
floodrisk_100_year_url_Regional = 'https://gis.water.ca.gov/arcgis/rest/services/Boundaries/BAM/MapServer/7' #Regional/Special Studies
floodrisk_100_year_url_DWR = 'https://gis.water.ca.gov/arcgis/rest/services/Boundaries/BAM/MapServer/8' #DWR Awareness
floodrisk_100_year_url_USACE = 'https://gis.water.ca.gov/arcgis/rest/services/Boundaries/BAM/MapServer/9' #USACE Comprehensive Study


In [8]:
#Read the data in as feature layers
floodrisk_100yearlayer1 = FeatureLayer(floodrisk_100_year_url_FEMA)
floodrisk_100yearlayer2 = FeatureLayer(floodrisk_100_year_url_Regional)
floodrisk_100yearlayer3 = FeatureLayer(floodrisk_100_year_url_DWR)
floodrisk_100yearlayer4 = FeatureLayer(floodrisk_100_year_url_USACE)

In [9]:
#import arcpy

#Add code to merge or dissolve layers into one

### Importing and combining 200 year flood plain data

In [ ]:
#Repeat for 200 year flood plain data

### Importing and combining 500 year flood plain data

In [ ]:
#Repeat for 500 year flood plain data

### Convert to spatial dataframes and wrangle data

In [11]:
#Convert layers to spatial dataframes
sdf_floodrisk_100year = GeoAccessor.from_layer(floodrisk_100yearlayer1)
#sdf_floodrisk_200year = GeoAccessor.from_layer(floodrisk_200yearlayer)
#sdf_floodrisk_500year = GeoAccessor.from_layer(floodrisk_500yearlayer)

In [12]:
sdf_floodrisk_100year.columns

Index(['OBJECTID', 'DFIRM_ID', 'VERSION_ID', 'FLD_AR_ID', 'STUDY_TYP',
       'FLD_ZONE', 'ZONE_SUBTY', 'SFHA_TF', 'STATIC_BFE', 'V_DATUM', 'DEPTH',
       'LEN_UNIT', 'VELOCITY', 'VEL_UNIT', 'AR_REVERT', 'AR_SUBTRV',
       'BFE_REVERT', 'DEP_REVERT', 'DUAL_ZONE', 'SOURCE_CIT', 'GFID',
       'Shape_Length', 'Shape_Area', 'CO_FIPS', 'County', 'SHAPE'],
      dtype='object')

In [13]:
sdf_floodrisk_100year.head()

,OBJECTID,DFIRM_ID,VERSION_ID,FLD_AR_ID,STUDY_TYP,FLD_ZONE,ZONE_SUBTY,SFHA_TF,STATIC_BFE,V_DATUM,...,BFE_REVERT,DEP_REVERT,DUAL_ZONE,SOURCE_CIT,GFID,Shape_Length,Shape_Area,CO_FIPS,County,SHAPE
0,1,06087C,1.1.1.0,06087C_6,NP,A,<NA>,T,-9999.0,<NA>,...,-9999.0,-9999.0,<NA>,06087C_STUDY1,5c32f5e6-e407-499e-9aac-21070df175a9,2480.178083,40344.798174,087,Santa Cruz,"{""rings"": [[[-189089.44590000063, -93566.90960..."
1,2,06087C,1.1.1.0,06087C_12,NP,A,<NA>,T,-9999.0,<NA>,...,-9999.0,-9999.0,<NA>,06087C_STUDY1,5c32f5e6-e407-499e-9aac-21070df175a9,724.180157,24847.320406,087,Santa Cruz,"{""rings"": [[[-160979.05110000074, -115433.7480..."
2,3,06087C,1.1.1.0,06087C_14,NP,A,<NA>,T,-9999.0,<NA>,...,-9999.0,-9999.0,<NA>,06087C_STUDY1,5c32f5e6-e407-499e-9aac-21070df175a9,1407.449224,137581.273971,087,Santa Cruz,"{""rings"": [[[-143286.5594999995, -122125.10710..."
3,4,06087C,1.1.1.0,06087C_21,NP,A,<NA>,T,-9999.0,<NA>,...,-9999.0,-9999.0,<NA>,06087C_STUDY1,5c32f5e6-e407-499e-9aac-21070df175a9,15655.534418,902558.705077,087,Santa Cruz,"{""rings"": [[[-182445.5996000003, -96833.979199..."
4,5,06087C,1.1.1.0,06087C_62,NP,A,<NA>,T,-9999.0,<NA>,...,-9999.0,-9999.0,<NA>,06087C_STUDY1,5c32f5e6-e407-499e-9aac-21070df175a9,1057.795507,19269.758301,087,Santa Cruz,"{""rings"": [[[-177217.72959999926, -114214.1187..."


### Import county and roads shapefiles: could potentially enrich data to apply data engineering principles

In [ ]:
#Code to import boundary and roads data

### Import electrical infrastructure data: transmission lines, substations, power plants and EV charging stations

In [ ]:
#Code to import electrical infrastructure

### Import CalAdapt data by year - Allow user to specify and specific year and import the relevant shapefile in the time series

In [ ]:
#Code to import wildfire, sea level rise and extreme heat data

### Find out which assets are affected by extreme weather events

We could find the intersection and create a dataframe with a column for derating and put these in a dashboard showing number of assets affected and severity.

### Find out which transmission lines and EV chargers might be inaccessible/offline following applicable extreme weather events

We could apply the same approach as the Puerto Rico paper and do a special analysis for EV chargers to show how distance to an working charger might be impacted across the different counties.

### Export shapefiles to be visualized in dashboard